In [ ]:
import random
import numpy as np

random.seed(42)

def make_distances(city_count):
    dists = [[0 if i == j else random.randint(3, 50) for j in range(city_count)] for i in range(city_count)]
    for i in range(city_count):
        for j in range(i + 1, city_count):
            dists[j][i] = dists[i][j]
    return dists

dist10 = make_distances(10)
dist20 = make_distances(20)

print("Distances for 10 cities:")
for row in dist10:
    print(row)

print("\nDistances for 20 cities:")
for row in dist20:
    print(row)

class ACO:
    def __init__(self, dist, ants, iterations, alpha=1.0, beta=5.0, evap=0.5, Q=100):
        self.dist = np.array(dist)
        self.pher = np.ones(self.dist.shape) / len(dist)
        self.cities = range(len(dist))
        self.ants = ants
        self.iter = iterations
        self.alpha = alpha
        self.beta = beta
        self.evap = evap
        self.Q = Q

    def solve(self):
        best_path = None
        best_len = float('inf')

        for _ in range(self.iter):
            paths = self.build_paths()
            self.update_pheromones(paths)
            for path, length in paths:
                if length < best_len:
                    best_len = length
                    best_path = path
            self.pher *= (1 - self.evap)

        return best_path, best_len

    def build_paths(self):
        result = []
        for _ in range(self.ants):
            path = []
            seen = set()
            curr = random.randint(0, len(self.dist) - 1)
            seen.add(curr)
            path.append(curr)
            while len(seen) < len(self.dist):
                nxt = self.pick_next(curr, seen)
                path.append(nxt)
                seen.add(nxt)
                curr = nxt
            path.append(path[0])
            length = self.total_length(path)
            result.append((path, length))
        return result

    def pick_next(self, curr, seen):
        probs = []
        pher = self.pher[curr]
        vis = 1 / (self.dist[curr] + 1e-10)
        for i in self.cities:
            if i in seen:
                probs.append(0)
            else:
                probs.append((pher[i] ** self.alpha) * (vis[i] ** self.beta))
        total = sum(probs)
        probs = [p / total for p in probs]
        return np.random.choice(self.cities, p=probs)

    def total_length(self, path):
        s = 0
        for i in range(len(path) - 1):
            s += self.dist[path[i]][path[i+1]]
        return s

    def update_pheromones(self, paths):
        for path, length in paths:
            for i in range(len(path) - 1):
                a = path[i]
                b = path[i + 1]
                self.pher[a][b] += self.Q / length
                self.pher[b][a] += self.Q / length

def run_case(dists, ants_list):
    for count in ants_list:
        print(f"\nUsing {count} ants:")
        algo = ACO(dists, ants=count, iterations=50)
        path, cost = algo.solve()
        print("Best path found:", path)
        print("Total cost:", cost)

print("\n--- Results for 10 cities ---")
run_case(dist10, [1, 5, 10, 20])

print("\n--- Results for 20 cities ---")
run_case(dist20, [1, 5, 10, 20])

Distances for 10 cities:
[0, 43, 10, 4, 50, 20, 18, 17, 11, 50]
[43, 0, 46, 50, 37, 8, 40, 30, 5, 4]
[10, 46, 0, 17, 35, 41, 4, 38, 15, 48]
[4, 50, 17, 0, 29, 17, 31, 40, 20, 3]
[50, 37, 35, 29, 0, 20, 12, 16, 24, 9]
[20, 8, 41, 17, 20, 0, 41, 19, 5, 49]
[18, 40, 4, 31, 12, 41, 0, 21, 43, 42]
[17, 30, 38, 40, 16, 19, 21, 0, 17, 21]
[11, 5, 15, 20, 24, 5, 43, 17, 0, 13]
[50, 4, 48, 3, 9, 49, 42, 21, 13, 0]

Distances for 20 cities:
[0, 41, 43, 13, 37, 49, 18, 13, 32, 27, 20, 43, 47, 38, 17, 46, 23, 6, 17, 5]
[41, 0, 28, 20, 7, 16, 39, 48, 23, 16, 44, 34, 28, 44, 32, 12, 19, 11, 18, 50]
[43, 28, 0, 19, 50, 40, 30, 40, 28, 26, 17, 11, 35, 34, 8, 6, 10, 12, 43, 13]
[13, 20, 19, 0, 7, 27, 27, 41, 32, 36, 19, 38, 3, 46, 49, 10, 46, 37, 20, 44]
[37, 7, 50, 7, 0, 13, 32, 3, 49, 49, 19, 35, 14, 35, 9, 43, 22, 43, 35, 41]
[49, 16, 40, 27, 13, 0, 36, 3, 41, 23, 34, 4, 10, 26, 22, 18, 6, 18, 39, 8]
[18, 39, 30, 27, 32, 36, 0, 11, 45, 33, 38, 13, 19, 36, 41, 30, 16, 37, 49, 47]
[13, 48, 40, 41, 3, 